# Cyrite HDL: Guide for migration from MyHDL

These are some crucial differences to the MyHDL internals when migrating towards cyHDL. The `cyhdl` layer with its built-in generation syntax will force you to follow a little different way of thinking.

To outline the paradigm shifts:

* Modular translation instead of flattened hierarchy: May require strict types (see below)
* MyHDL code `@block` components are AST translated to an intermediate Python notation ('IRL')
* IRL is 'transpiled' to the target HDL (executed) from myIRL, unlike 'translated' (by AST-transformation)
* Conversion and implementation rules may be buried the extension objects (`@blackbox`es, custom signals, etc.)
* Stricter interface rules (**Type annotation**), less implicit handling of mixed type parameters (constants, signals)
* Arithmetics: the MyHDL datatype `intbv` is inherited from MyHDL almost 'as is' and its arithmetic behaviour is identical in simulation. However, some statements may require explicit casting when signed conversions are involved. So, the emulation framework may be more strict in some cases and enforce some code discipline according to particular design rules.

Import in emulation mode:

In [1]:
from cyhdl import *

## Interface

The interface represents the port/parameter layer connecting hierarchical modules, which is distilled from the function parameters during analysis. However, not all function parameters infer to an interface (more below).

Type annotation is optional, but highly recommended for signals routed through the hierarchy. Missing annotation will now result in a warning.

You may still pass down either a constant or a signal via a classic port parameter and omit the annotation ('legacy' myHDL style). However, this can have unwanted side effects. For example, it may require you to explicitely `.insert()` a signal into its parenting function, if it ends up undefined in the resulting HDL during conversion. This behaviour may change in later releases.

Type annotation may also come into play to explicitely infer a specific interface in the resulting HDL.

For the code below, `s` can only be a `bool()` or Signal of it. To avoid warnings, we don't specify a type annotation for `s`.

In [2]:
@block
def unit(a : Signal, q : Signal.Output, s):
    @always_comb
    def worker():
        if s: # This is allowed with a bool or bit signal
            q.next = a
        else:
            q.next = ~a
    
    return locals()

Note also the `locals()` call instead of `instances()` (which is in fact now identical within a Python context). This allows some more analysis (signals, etc.) in the future. It is **required** to use the `locals()` notation within a cythonizeable (compiled) module.

In [3]:
from myirl.test.common_test import run_ghdl, SYSTEM_DIR

def testx(uut, MODE = 0):
    a, q = [ Signal(bool())  for i in range(2) ]
    if MODE:
        s = Signal(bool())
    else:
        s = False
    
    inst = uut(a, q, s)
    return inst

inst = testx(unit)
f0 = inst.elab(targets.VHDL)

 Writing 'unit' to file /tmp/myirl_unit_5g72z6bd/unit.vhdl 


For details of the interface generation, see IRL details [Interfaces](interfaces.ipynb).

### Strict wire type interface

The strict wire type is created by a `Signal.Type` invocation. When a type annotation is a tuple of types, arguments are checked against each item and the first match wins.

In [4]:
Bool = Signal.Type(bool)

@block
def unit_s(a : Bool, q : Bool.Output, s : (Bool, bool)):
    @always_comb
    def worker():
        if s: # This is allowed with a bool or bit signal
            q.next = a
        else:
            q.next = ~a
    
    return locals()

inst = testx(unit_s)
f0 = inst.elab(targets.VHDL)

 Writing 'unit_s' to file /tmp/myirl_unit_s_8nipb0sf/unit_s.vhdl 


### Differences in VHDL output

We output both scenarios and run the resulting VHDL code through analysis plus `diff` to see how the interface is inferred:

In [5]:
run_ghdl(f0, inst, debug = False)

0

In [6]:
inst = testx(unit_s, 1)
f1 = inst.elab(targets.VHDL)
run_ghdl(f1, inst)

 Module unit_s: Existing instance unit_s, rename to unit_s_1 
 Writing 'unit_s_1' to file /tmp/myirl_unit_s_hazfu0ly/unit_s_1.vhdl 


0

In [7]:
! diff --label unit --label unit_1 --color=always -u {f0[0]} {f1[0]}

--- unit
+++ unit_1
@@ -12,22 +12,23 @@
 use work.txt_util.all;
 use work.myirl_conversion.all;
 
-entity unit_s is
+entity unit_s_1 is
     port (
         a : in std_ulogic;
-        q : out std_ulogic
+        q : out std_ulogic;
+        s : in std_ulogic
     );
-end entity unit_s;
+end entity unit_s_1;
 
-architecture cyriteHDL of unit_s is
+architecture cyriteHDL of unit_s_1 is
     -- Local type declarations
     -- Signal declarations
 begin
     
 worker:
-    process(a)
+    process(a, s)
     begin
-        if FALSE then
+        if (s = '1') then
             q <= a;
         else
             q <= not a;


### Interface types

Only `Signal` type derivatives will infer to a port. All others pass as 'run time' parameters, unless they are explicitely passed as parameter, specified past the PEP570 `*` delimiter. This has the following effects:

* Bare (undecorated) class support is experimental and translates Signals members, only.
* A bare class unrolls into members and in/out direction is guessed
* `dict` or `tuple` container types of signals will create local instances of their children

In [8]:
@block
def unit_local_inst(d : PassThrough(dict), t : PassThrough(tuple)):
    
    a = d['a']
    b = d['b']
    
    q = t[0]
    
    s = concat(a, b)

    @always_comb
    def worker():
        q.next = s
    
    return instances()

To explicitely pass through an argument without warning, we use the PassThrough() type construct. This is recommended practise.

In [9]:
def convert(uut):
    a, b = [ Signal(intbv(0x5)[4:], name=n) for n in ('a', 'b') ]
    q = Signal(intbv()[8:], name='q')
    inst = uut({'a' : a, 'b' : b}, (q,))
    return inst.elab(targets.VHDL)
f = convert(unit_local_inst)

 Writing 'unit_local_inst' to file /tmp/myirl_unit_local_inst__705fvl7/unit_local_inst.vhdl 


In [10]:
! grep -A 10 declarations {f[0]}

    -- Local type declarations
    -- Signal declarations
    signal q : unsigned(7 downto 0);
    signal a : unsigned(3 downto 0);
    signal b : unsigned(3 downto 0);
begin
    
worker:
    process(a, b)
    begin
        q <= (a & b);
    end process;


### Inferred port signal types

The hierarchical interfaces are restricted to signals with a well defined width, as if they were physical pins.

Therefore, the following data types can not be used as `in`/`out` signal arguments:

* Vector/Array data types
* Other custom types not derived from the `Signal` class.

### Type checking

Errors will now be raised on type mismatches. See [Type checking](typechecking.ipynb) for details.

## Loops

Loops are no longer supported *inside* logic generators, except `@instance` functions that generate sequential statements.

Support is currently rather limited to the constructs below.

### Iteration and range

In [11]:
@block
def tb_for_loop():
    s, a = [ Signal(intbv()[4:]) for _ in range(2) ]

    @always_comb
    def worker():
        s.next = a + 1

    @instance
    def stim():
        for j in range(3):
            a.next = j
            yield delay(1)
            assert s == (j + 1)

        for it in [ (0, 1), (12, 13) ]:
            a.next = it[0]
            yield delay(1)
            assert s == it[1]

    return instances()


def test_for_loop():
    uut = tb_for_loop()
    f = uut.elab(targets.VHDL)
    run_ghdl(f, tb_for_loop)

test_for_loop()

 Writing 'tb_for_loop' to file /tmp/myirl_tb_for_loop_qt05h6eb/tb_for_loop.vhdl 


## Variables

Variables in general have shown to be cumbersome in HDL translation and are causes for many pitfalls and confusing debug scenarios. Therefore: handle with care, **try avoiding variables in the HDL generation** where possible. For RTLIL direct synthesis, Variable types are **no longer** supported.

However, variables support is maintained for the **test bench** (`@simulation.generator`) IRL functionality. As of now, IRL and myHDL constructs can be mixed.

To define a variable 'myhdl style', the first assignment in the local sub function counts and defines the type (which can not be altered). Therefore, logical expressions are not valid for first time assignment.

**Note** : Variable support is incomplete:
 * Do not use booleans for std_logic synthesis

In [12]:
@block
def tb_var():
    @instance
    def seq():
        v = intbv(10)[5:] # Intbv type
        z = -1             # Unlimited integer
        s = "LUT"
        
    return instances()

Conversion to VHDL:

In [13]:
def convert(uut):
    inst = uut()
    files = inst.elab(targets.VHDL)
    return files

f = convert(tb_var)

 Writing 'tb_var' to file /tmp/myirl_tb_var_l2cxbvmc/tb_var.vhdl 


And resulting output:

In [14]:
!grep -A 10 process {f[0]}

    process
    variable v : unsigned(4 downto 0);
    variable z : integer;
    variable s : string;
    begin
        v := "01010";
        z := -1;
        s := "LUT";
        wait;
    end process;
end architecture cyriteHDL;



For further verification using GHDL, we insert an auxiliary:

In [15]:
def test_top(uut):
    inst = top(uut)
    f = inst.elab(targets.VHDL, elab_all=True)
    run_ghdl(f, inst, analyze_only = True)
    return f

def test_unit(uut, debug = False):
    inst = uut()
    vhdl93 = targets.vhdl.VHDL93()
    f = inst.elab(vhdl93, elab_all=True)
    run_ghdl(f, inst, debug = debug, std = '93')
    return f

## Signals

The `components.Signal` class emulates the myHDL signal with the native `intbv` wire type behaviour up to a few corner cases which are handled differently (in particular with arithmetics).

However, there are new signal types introduced:
* ClkSignal: A dedicated clock signal

From the top level, a clock signal will require redefining to `ClkSignal()`.

See more about signal types in [Signals](../notebooks/signals.ipynb).

## Logic operations

In myHDL, you can more or less freely treat boolean signals, logical combinations and mixtures of those the same.
However, the internal representation reduces all that to binary logic operations, therefore legacy myHDL style is translated accordingly.

This is tricky, as coding style previously had an impact to the translation to various VHDL standards, for instance, VHDL-08 is more relaxed with respect to boolean and signal treatment.

**Note:** *Boolean* combinations are allowed only within conditional statements. When assigning logical combinations to a signal, you *must* use a bitwise operator.

See also [myIRL reference on operations](../notebooks/operations.ipynb).

In [16]:
@block
def tb_ck():
    clk = ClkSignal()
    
    @always(delay(1))
    def clkgen():
        clk.next = not clk
        
    return instances()

try:
    test_unit(tb_ck)
except ValueError as e:
    print("EXPECTED", e)

 Writing 'tb_ck' to file /tmp/myirl_tb_ck_lnsraksv/tb_ck.vhdl 
 Creating library file module_defs.vhdl 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/emulation/myhdl2irl.py:741: UserWarning: /tmp/ipykernel_14293/4270055741.py:tb_ck():7 Replacing logical `not` by inversion
  warnings.warn(self.get_location(node) + \


On a sidenote: see the translation to IRL of the above tb_ck unit:

In [17]:
print(tb_ck.unparse())

Unparsing unit tb_ck


@block
def tb_ck():
    clk = ClkSignal()

    @always_(delay(1))
    def clkgen(_context):
        (yield [clk.set((~ clk))])
    return instances()



The following unit translates a condition using boolean logic as follows:

In [18]:
@block
def unit():
    a = Signal(intbv(0xaa)[8:])
    a.init = True
    q = Signal(bool())

    @instance
    def stim():
        q.next = False
        if a[0] == True and a[1] == False and a[7] == False:
            q.next = True
            
        yield delay(1)
        assert q == False
    
    return instances()

test_unit(unit)

 Writing 'unit' to file /tmp/myirl_unit_we3v48g3/unit.vhdl 
 Creating library file module_defs.vhdl 


['/tmp/myirl_unit_we3v48g3/unit.vhdl', 'module_defs.vhdl']

Booleans for variables are also implemented:

In [19]:
@block
def unit_var():
    a = Signal(intbv(0xaa)[8:])
    a.init = True
    q = Signal(bool())

    @instance
    def stim():
        v = False
        v = (a[0] == True) & (a[1] == False) & (a[7] == False)
        q.next = False
        yield delay(1) # Must wait to settle
        if v: # Make sure to not use `v == True`:
            q.next = True
            
        yield delay(1)
        assert q == False
    
    return instances()
try:
    test_unit(unit_var, debug = False)
except Exception as e:
    print("Expected failure", e)

 Writing 'unit_var' to file /tmp/myirl_unit_var_75j_xeh7/unit_var.vhdl 
 Creating library file module_defs.vhdl 


However when you use a reference instead of a variable, outside the logic generator, avoiding comparison functions:

In [20]:
@block
def unit_ref():
    a = Signal(intbv(0xaa)[8:])
        
    a.init = True
    q = Signal(bool())
    
    v = (a[0]) & (~a[1]) & (a[7]) # compact bit wise operations

    @instance
    def stim():
        q.next = False
        yield delay(1) # Must wait to settle
        if v == True:
            q.next = True
            
        yield delay(1)
        assert q == False
    
    return instances()

test_unit(unit_ref)

 Writing 'unit_ref' to file /tmp/myirl_unit_ref_2kc7g8uk/unit_ref.vhdl 
 Creating library file module_defs.vhdl 


['/tmp/myirl_unit_ref_2kc7g8uk/unit_ref.vhdl', 'module_defs.vhdl']

### Shadow signals

Slicing `ShadowSignal` instances are no longer necessary and are considered deprecated. Currently, they are emulated with a warning.

In [21]:
a = Signal(intbv(0xaa)[8:])
sa = a(4,0)

/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/emulation/signals.py:16: DeprecatedWarning: Deprecated indexing syntax, use `[]` instead
  warnings.warn(*args, category)


In [22]:
sa.evaluate()

intbv(10)

**Note**: ShadowSignal emulations will not work fully throughout the hierarchy and are for now supported on local signals only

### Concatenation

The `concat()` function can be used in the behavioural as well as the structural section of the `@block`.

In [23]:
from myirl.emulation.signals import ConcatSignal # deprecated

@block
def unit(a : Signal, b : Signal, q : Signal.Output, MODE=0):
    
    if MODE:
        s = concat(a, b)
    else:
        s = ConcatSignal(a, b)
    
    @always_comb
    def worker():
        q.next = s
    
    return instances()

In [24]:
def convert(uut):
    a, b = [ Signal(intbv(0x5)[4:], name=n) for n in ('a', 'b') ]
    q = Signal(intbv()[8:], name='q')
    inst = uut(a, b, q, MODE=0) # Change mode to '1' to make the warning disappear
    return inst

inst = convert(unit)
f = inst.elab(targets.VHDL)

 Writing 'unit' to file /tmp/myirl_unit_p_ademe4/unit.vhdl 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/emulation/signals.py:16: DeprecatedWarning: Deprecated ConcatSignal() syntax, use concat() instead
  warnings.warn(*args, category)


The resulting process in VHDL:

In [25]:
! grep -A 5 worker {f[0]}

worker:
    process(a, b)
    begin
        q <= (a & b);
    end process;



# Functions

Functions are no longer translated to HDL, but executed. Therefore, you'll have to implement your previous function as a `@blackbox` or `@hdlmacro` entity if you wish to specifically generate hardware, or transfer the functionality into a HDL block. Very simple operators can be also implemented by derival of `base.BinOp`, etc.

### Functional logic generation

These undecorated variants will just be called and unroll in the resulting transfer representation (such as VHDL). As an example, a full adder function can be instanced as follows:

In [26]:
def full_adder0(a, b, cin, q, c):
    x = a ^ b
    return [
        q.set(x ^ cin), c.set((a & b) | (cin & x))
    ]

In the resulting VHDL, this will unroll as:

In [27]:
@block
def simple_adder0():
    a, b = [ Signal(intbv()[2:], name = n) for n in "ab" ]
    c = Signal(bool())
    q = [ Signal(bool()) for _ in range(3)]
    result = Signal(intbv()[3:], name = "res")
    
    wires = [
        *full_adder0(a[0], b[0], False, q[0], c),
        *full_adder0(a[1], b[1], c, q[1], q[2]),
        result.set(concat(*reversed(q)))
    ]
    
    return instances()

f = test_unit(simple_adder0, debug = False)

 Writing 'simple_adder0' to file /tmp/myirl_simple_adder0_rp2vm6xc/simple_adder0.vhdl 
 Creating library file module_defs.vhdl 


In [28]:
! grep -A 10 begin {f[0]}

begin
    q0 <= (a(0) xor b(0));
    c <= ((a(0) and b(0)) or '0');
    q1 <= ((a(1) xor b(1)) xor c);
    q2 <= ((a(1) and b(1)) or (c and (a(1) xor b(1))));
    res <= (q2 & q1 & q0);
end architecture cyriteHDL;



### HDL macros

A more elegant way is to decorate the function with `@hdlmacro` to avoid the `*` notation. Note that the hardware elements are returned using `yield`.

In [29]:
@hdlmacro
def full_adder(a, b, cin, q, c):
    x = a ^ b
    yield [
        q.set(x ^ cin), c.set((a & b) | (cin & x))
    ]

In [30]:
@block
def simple_adder():
    a, b = [ Signal(intbv()[2:], name = n) for n in "ab" ]
    c = Signal(bool())
    q = [ Signal(bool()) for _ in range(3)]
    result = Signal(intbv()[3:], name = "res")
    
    wires = [
        full_adder(a[0], b[0], False, q[0], c),
        full_adder(a[1], b[1], c, q[1], q[2]),
        result.set(concat(*reversed(q)))
    ]
    
    return instances()

f = test_unit(simple_adder, debug = False)

 Writing 'simple_adder' to file /tmp/myirl_simple_adder_jgvv7z73/simple_adder.vhdl 
 Creating library file module_defs.vhdl 


**Note:** `@hdlmacro` functions in sequential simulation specific code may not be portable to co-simulation test benches. It is recommended to use this construct for hardware specific descriptions only.

### Function wrappers

Example of a `reduceOr` pseudo function that can resolve into different implementations, depending on the HDL target:

In [31]:
from myirl.library.reduce import reduceOr

@block
def unit1(a : Signal, q : Signal.Output):
    @always_comb
    def assign():
        q.next = reduceOr(a)
        
    return instances()

In [32]:
def convert(uut):
    a = Signal(intbv("1010100"))
    q = Signal(bool())
    inst = uut(a, q)
    return inst

inst = convert(unit1)
f = inst.elab(targets.VHDL)

 Writing 'unit1' to file /tmp/myirl_unit1_f4s6xi0f/unit1.vhdl 
DEBUG Type descriptor [function 'reduce_or/reduce_or'] not owned by a lib. Skipping <class 'myirl.kernel.components.function'>


In [33]:
! grep -A 8 begin {f[0]}

begin
    
assign:
    process(a)
    begin
        q <= or_reduce(STD_LOGIC_VECTOR(a));
    end process;

end architecture cyriteHDL;



## Member assignment

Constructs like `s.next[i] = val` are no longer supported.

The reasons:
   * The `next[i]` construct does no longer translate simply into a proper setter. Problematic in Co-Simulation.
   * It complicates direct synthesis a lot, as particular driver tables need to be maintained for signal members
   * It is simply considered unlogic notation as opposed to `s[i].next`

    
It can be emulated using variables as shown below. (This particular example is somewhat bloated, as the `v` elements are used nowhere else). However, this notation is not portable and is only tested for VHDL transfer.

In [34]:
@block
def unit_swap_bits(a : Signal, q : Signal.Output):
    @always_comb
    def worker():
        v = intbv()[2:]

        v[0] = a[1]
        v[1] = a[0]
        
        q.next = v
    
    return instances()

@block
def top(uut):
    a, b = [ Signal(intbv()[2:], name=n) for n in ('a', 'b') ]
    
    inst = uut(a, b)
    return instances()

print(unit_swap_bits.unparse())

Unparsing unit unit_swap_bits


@block
def unit_swap_bits(a: Signal, q: Signal.Output):

    @always_comb_
    def worker(_context):
        v = Variable('v', intbv()[2:])
        (yield [v.assign(intbv()[2:]), ElemAssign(v, 0, a[1]), ElemAssign(v, 1, a[0]), q.set(v)])
    return instances()



In [35]:
f = test_top(unit_swap_bits)

 Writing 'unit_swap_bits' to file /tmp/myirl_top__7zyly5m/unit_swap_bits.vhdl 
 Writing 'top' to file /tmp/myirl_top__7zyly5m/top.vhdl 
 Creating library file module_defs.vhdl 


In [36]:
! grep -A 8 worker {f[0]}

worker:
    process(a)
        variable v : unsigned(1 downto 0);
    begin
        v := "00";
        v(0) := a(1);
        v(1) := a(0);
        q <= v;
    end process;


A nicer/better coding style would be the snippet below (assuming the single bit `t` array signals would be used somewhere else, if not, we'd get by without them using `BLOAT = False`). 
**Note**: concat arguments are MSB..LSB order (which does the swapping the elegant way)

In [37]:
@block
def unit_swap_bits1(a : Signal, q : Signal.Output, BLOAT = True):
    if BLOAT:
        t = [ Signal(bool()) for i in range(2) ]

        a0 = t[0].wireup(a[0]) # Create wire instance
        a1 = t[1].wireup(a[1])
        wiring = q.wireup(concat(*t)) # Note: this is in fact `concat(t[0], t[1])
    else:
        wiring = q.wireup(concat(a[0], a[1]))

    return instances()

In [38]:
f = test_top(unit_swap_bits1)

 Module top: Existing instance top, rename to top_1 
Using default for BLOAT: True
 BLOAT: use default True 
 Writing 'unit_swap_bits1' to file /tmp/myirl_top_a_dbocrm/unit_swap_bits1.vhdl 
 Writing 'top_1' to file /tmp/myirl_top_a_dbocrm/top_1.vhdl 
 Creating library file module_defs.vhdl 


In [39]:
! grep -A 4 begin {f[0]}

begin
    t0 <= a(0);
    t1 <= a(1);
    q <= (t0 & t1);
end architecture cyriteHDL;


If member assignment is necessary, a different (derived) Signal type has to be used. See [Member assignment](../notebooks/member_assignment.ipynb).

In general, cyrite HDL coding considers member assignment as deprecated, as it can lead into all sort of confusing scenarios or undesired effects and mismatches between synthesis and simulation.
In most design cases, partial assignment is only used in memory mapped register decoding. This is handled by the cyrite library on a higher design
abstraction layer.
See [Register assignments](registers.ipynb) for the proper way to portable encode partial signal assignments. This also gives you full, switcheable control over how undefined or don't care signals are handled internally.

## Class signals

Basic legacy class is supported, however, non decorated classes unroll into their signal members while input and outputs are guessed from their driven or consumed state.

See also [Classes](../notebooks/classes.ipynb) for internal details.
Basically, you can wrap a class using the `@container` construct, taking a type generator class as argument. In myHDL emulation, this is represented by the `LEGACY_CLASS`, internally identical with `myirl.library.typegen.BareClass`. The type generator defines, how the signal class is unrolled in the resulting HDL or RTL representation.

In [40]:
@container(CONTAINER_LEGACY)
class SignalContainer:
    def __init__(self):
        self.a, self.b = [ Signal(bool()) for _ in range(2) ]

In [41]:
@block
def unit(c : SignalContainer, a : Signal.Output, b : Signal.Output):
    @always_comb
    def worker():
        b.next = c.a
        a.next = c.b
    return instances()

@block
def tb():
    b0, b1 = [ Signal(bool()) for _ in range(2) ]
    c = SignalContainer()

    inst = unit(c, b0, b1)
    return instances()

def test():
    t = tb()
    f = t.elab(targets.VHDL, elab_all = True)
    return f
    
f = test()

 Writing 'unit' to file /tmp/myirl_tb_ka6lilvp/unit.vhdl 
 Writing 'tb' to file /tmp/myirl_tb_ka6lilvp/tb.vhdl 
 Creating library file module_defs.vhdl 


In [42]:
!grep -A 6 port {f[0]}

    port (
        c_a : in std_ulogic;
        c_b : in std_ulogic;
        a : out std_ulogic;
        b : out std_ulogic
    );
end entity unit;


In [43]:
!grep -A 10 worker {f[0]}

worker:
    process(c_a, c_b)
    begin
        b <= c_a;
        a <= c_b;
    end process;

end architecture cyriteHDL;



### Bidirectional interface classes

Classes with inputs, outputs and special auxiliary signals use the `@container()` decorator without argument. However, they require special class members denoting input, output and auxiliar (other) signal member names. Signal names can *not* start with an underscore.

In [44]:
@container(CONTAINER_INTERFACE)
class BidirSignalContainer:
    _inputs = ['a']
    _outputs = ['b']
    _other = []
    
    def __init__(self):
        self.a, self.b = [ Signal(bool()) for _ in range(2) ]

When migrating from legacy classes, keep in mind that the new API is stricter with input/output handling when resolving the hierarchy.
Therefore you may run into errors when a class signal designated as output is being read from.

If a `container()` class is used in the interface, a structure/record is created for each `_inputs`, `_outputs` and `_other` set. If a unit requires the reverse direction (for example, being a master instead of a slave), you must annotate the type accordingly using `BidirSignalContainer.reverse()`.

## Conditional translation, parameters

A `@block` unit that uses configuration variables to generate hardware dependent on their state should take these configuration values as a parameter, if this unit is to be used several times in different configurations within the design. The reason is that a signature is created from the interface such that equal configurations are not re-created. When using a global variable for conditional translation, this will fail.

## Constructs with issues

This is a list of constructs that has or has had issues.

### Partially fixed: Augmented assign to list of signals

In [45]:
@block
def augassign_fixed():
    worker = [ intbv(0)[1:] ]
    worker += [ Signal(intbv(i)[2:]) for i in range(3) ]
    # worker = worker + [ Signal(intbv(i)[2:]) for i in range(3) ]
    
    @instance
    def stim():
        # Note: Can't replace worker[0]:
        # worker[0][:] = 0
        worker[1].next = worker[0]
        worker[2].next = 1
    return locals()

print(augassign_fixed.unparse())

Unparsing unit augassign_fixed


@block
def augassign_fixed():
    worker = [intbv(0)[1:]]
    worker += [Signal(intbv(i)[2:]) for i in range(3)]

    @generator_ctx
    def stim(_context):
        (yield [worker[1].set(worker[0]), worker[2].set(1)])
    return locals()



### Fixed: Top level creation

Earlier versions did not allow instancing in global namespace (top level):

In [46]:
u = augassign_fixed()
f = u.elab(targets.VHDL)

 Writing 'augassign_fixed' to file /tmp/myirl_augassign_fixed_4ml3e8t_/augassign_fixed.vhdl 


### Legacy Delayed signals

Delays are no longer a basic signal class property. If a particular delay behaviour is desired, a custom class should be derived from the `Signal` class.
The internal `SigAssign` generator has notion of a delay specification, however this is not a portable construct, i.e. its behaviour is target specific and should not be used from a synthesizeable design at all.

**This is a deprecated approach and is no longer supported**

For some simulation specific `DelaySignal` classes, it is possible to use

```
s.set(a, delay = 2)
```

where the time unit is a property of the Signal class.

Unlike MyHDL, this delay specification propagates into the resulting HDL simulation target.

For simulation output to native HDLs such as Verilog or VHDL, the `simulation` API (not `myirl.simulation`) provides various methods:

* `@always(delay(m))` decorator: Delays all containing assignments by a given cycle respective delta time
* `simulation.delayed(expr, timeval)` : A unary operator with a given delta, causing emission of a delay statement

These statements only make sense within simulation specific HDL contexts. They are neither synthesizeable, nor portable.

The reason for not supporting the `delay` attribute is a cleaner separation between synthesis and simulation instances. It was also necessary to provide extended target language support.